### Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp

from scipy import stats

# Initialize seaborn (for plotting)
sns.set()

### Loading and inspecting data

In [2]:
# Depots
dfDepots = pd.read_csv("depots.csv", index_col=0)
dfDepots

,Cost,Capacity
Depot,,
1,14754,120
2,15814,140
3,9200,20
4,15870,120


In [3]:
# Demand Zones
dfZones = pd.read_csv("zones.csv", index_col=0)
dfZones

,Demand
Demand Zone,
1,82
2,41
3,94


In [4]:
# Zone-depot costs
dfZoneDepot = pd.read_csv("zone_depot_costs.csv")
dfZoneDepot

,Demand Zone,Depot,Cost
0,1,1,1998
1,2,1,1470
2,3,1,1993
3,1,2,1187
4,2,2,1217
5,3,2,1510
6,1,3,1005
7,2,3,1734
8,3,3,1100
9,1,4,1416


### Sets

In [5]:
# Depots
depots = list(dfDepots.index)
numdepots = len(depots)

# Demand zones
zones = list(dfZoneDepot["Demand Zone"].unique())
numzones = len(zones)

### Parameters

In [6]:
# Demand in each zone (dictionary)
zonedemand = {}
for j in zones:    
    zonedemand[j] = float(dfZones.loc[j]["Demand"])

# Depot capacities (dictionary)
depotcapacity = {}
for i in depots:
    depotcapacity[i] = int(dfDepots.loc[i]["Capacity"])

# Depot costs (dictionary)
depotcost = {}
for i in depots:
    depotcost[i] = float(dfDepots.loc[i]["Cost"])
    
# Depot-zone costs (dictionary)
depotzonecost = {}
for i in depots:
    for j in zones:
        depotzonecost[(i,j)] = float(dfZoneDepot[(dfZoneDepot["Depot"] == i)&(dfZoneDepot["Demand Zone"] == j)]["Cost"])

### Variables

In [7]:
# Variables: if a depot is opened/allocated
yvar = pulp.LpVariable.dict("y", depots, cat=pulp.LpBinary)

# Variables: amount of demand from each zone allocated to depot, per scenario
xvar = pulp.LpVariable.dict("x", (depots, zones), lowBound=0.0, cat=pulp.LpContinuous)

### Model initialization


In [8]:
# Initialize model and objective sense
locationModel = pulp.LpProblem(name="LocationModel", sense=pulp.LpMinimize)

### Constraints

In [9]:
# Contraint: demand must be satisfied in all scenarios
for j in zones:
    locationModel += pulp.lpSum( [xvar[(i,j)] for i in depots] ) == zonedemand[j]

# Constraint: depot capacities must be observed in all scenarios
for i in depots:        
    locationModel += pulp.lpSum( [xvar[(i,j)] for j in zones] ) <= depotcapacity[i] * yvar[i]

### Objective function

In [10]:
# Objective function

# --- depot allocation costs
obj = pulp.lpSum([ depotcost[i] * yvar[i] for i in depots])

# --- package flow costs
obj += pulp.lpSum( [ depotzonecost[(i,j)] * xvar[(i,j)] for i in depots for j in zones ] )

# add objective to model
locationModel += obj

### Solution process

In [11]:
# Write LP to file (optional, but often good to inspect model and find errors)
locationModel.writeLP("locationModel.lp")

# Solve model
locationModel.solve()
print("Status:", pulp.LpStatus[locationModel.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/cireandr/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/p6/k9kpfkxx7h78ybc6p8j_mp2m0000gp/T/d979ae40d7c241d89ff26ddbebc5a3f5-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/p6/k9kpfkxx7h78ybc6p8j_mp2m0000gp/T/d979ae40d7c241d89ff26ddbebc5a3f5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 65 RHS
At line 73 BOUNDS
At line 78 ENDATA
Problem MODEL has 7 rows, 16 columns and 28 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 323747 - 0.00 seconds
Cgl0004I processed model has 7 rows, 16 columns (4 integer (4 of which binary)) and 28 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.475
Cbc0038I Pass   1: suminf.    0.47500 (1) obj. 323747 iterations 0
Cbc0038I Solution found of 33207

In [12]:
# Total cost
totalCost = pulp.value(locationModel.objective)
print("Total cost: " + str(totalCost))

Total cost: 332079.0


In [13]:
# Print solution (you can add to the dataframe if needed)
for i in depots:
    if yvar[i].varValue >= 1.0:
        print("Depot " + str(i) + " is opened")                            
        for j in zones:
            if xvar[(i,j)].varValue > 0.0:
                print("\t\tServes zone " + str(j) + " - with capacity " + str(xvar[(i,j)].varValue))

Depot 2 is opened
		Serves zone 1 - with capacity 66.0
		Serves zone 3 - with capacity 74.0
Depot 3 is opened
		Serves zone 3 - with capacity 20.0
Depot 4 is opened
		Serves zone 1 - with capacity 16.0
		Serves zone 2 - with capacity 41.0
